In [37]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 13: Claw Contraption ---</h2><p>Next up: the <a href="/2020/day/24">lobby</a> of a resort on a tropical island. The Historians take a moment to admire the hexagonal floor tiles before spreading out.</p>
<p>Fortunately, it looks like the resort has a new <a href="https://en.wikipedia.org/wiki/Amusement_arcade">arcade</a>! Maybe you can win some prizes from the <a href="https://en.wikipedia.org/wiki/Claw_machine" target="_blank">claw machines</a>?</p>
<p>The claw machines here are a little unusual. Instead of a joystick or directional buttons to control the claw, these machines have two buttons labeled <code>A</code> and <code>B</code>. Worse, you can't just put in a token and play; it costs <em>3 tokens</em> to push the <code>A</code> button and <em>1 token</em> to push the <code>B</code> button.</p>
<p>With a little experimentation, you figure out that each machine's buttons are configured to move the claw a specific amount to the <em>right</em> (along the <code>X</code> axis) and a specific amount <em>forward</em> (along the <code>Y</code> axis) each time that button is pressed.</p>
<p>Each machine contains one <em>prize</em>; to win the prize, the claw must be positioned <em>exactly</em> above the prize on both the <code>X</code> and <code>Y</code> axes.</p>
<p>You wonder: what is the smallest number of tokens you would have to spend to win as many prizes as possible? You assemble a list of every machine's button behavior and prize location (your puzzle input). For example:</p>
<pre><code>Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400<br/>
Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176<br/>
Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450<br/>
Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279
</code></pre>

<p>This list describes the button configuration and prize location of four different claw machines.</p>
<p>For now, consider just the first claw machine in the list:</p>
<ul>
<li>Pushing the machine's <code>A</code> button would move the claw <code>94</code> units along the <code>X</code> axis and <code>34</code> units along the <code>Y</code> axis.</li>
<li>Pushing the <code>B</code> button would move the claw <code>22</code> units along the <code>X</code> axis and <code>67</code> units along the <code>Y</code> axis.</li>
<li>The prize is located at <code>X=8400</code>, <code>Y=5400</code>; this means that from the claw's initial position, it would need to move exactly <code>8400</code> units along the <code>X</code> axis and exactly <code>5400</code> units along the <code>Y</code> axis to be perfectly aligned with the prize in this machine.</li>
</ul>
<p>The cheapest way to win the prize is by pushing the <code>A</code> button <code>80</code> times and the <code>B</code> button <code>40</code> times. This would line up the claw along the <code>X</code> axis (because <code>80*94 + 40*22 = 8400</code>) and along the <code>Y</code> axis (because <code>80*34 + 40*67 = 5400</code>). Doing this would cost <code>80*3</code> tokens for the <span title="Half A presses are not allowed."><code>A</code> presses</span> and <code>40*1</code> for the <code>B</code> presses, a total of <code><em>280</em></code> tokens.</p>
<p>For the second and fourth claw machines, there is no combination of A and B presses that will ever win a prize.</p>
<p>For the third claw machine, the cheapest way to win the prize is by pushing the <code>A</code> button <code>38</code> times and the <code>B</code> button <code>86</code> times. Doing this would cost a total of <code><em>200</em></code> tokens.</p>
<p>So, the most prizes you could possibly win is two; the minimum tokens you would have to spend to win all (two) prizes is <code><em>480</em></code>.</p>
<p>You estimate that each button would need to be pressed <em>no more than <code>100</code> times</em> to win a prize. How else would someone be expected to play?</p>
<p>Figure out how to win as many prizes as possible. <em>What is the fewest tokens you would have to spend to win all possible prizes?</em></p>
</article>


In [44]:
from sympy import Matrix

tests = [
    {
        "name": "Small Example",
        "s": """
            Button A: X+94, Y+34
            Button B: X+22, Y+67
            Prize: X=8400, Y=5400

            Button A: X+26, Y+66
            Button B: X+67, Y+21
            Prize: X=12748, Y=12176

            Button A: X+17, Y+86
            Button B: X+84, Y+37
            Prize: X=7870, Y=6450

            Button A: X+69, Y+23
            Button B: X+27, Y+71
            Prize: X=18641, Y=10279
        """,
        "expected": 480,
    },
]


def fewest_tokens(s: str, delta: int = 0, debug=False) -> int:
    tokens = 0

    for i, line in enumerate(re.split(r"(?:\r?\n){2,}", s.strip()), start=1):
        x1, y1, x2, y2, x, y = (int(i) for i in re.findall(r"\d+", line))
        a = Matrix(
            [
                [x1, x2],
                [y1, y2],
            ],
        )
        b = Matrix([x + delta, y + delta])

        x = a.LDLsolve(b)

        if all(e.is_integer for e in x):
            if debug:
                print("\n".join((f"machine={i}", f"{a=}", f"{b=}", f"{x=}", "")))

            a, b = x
            tokens += 3 * a + b

    return tokens


@test(tests=tests)
def partI_test(s: str) -> int:
    return fewest_tokens(s)


Test Small Example passed, for partI_test.
Success


In [45]:
with open("../input/day13.txt") as f:
    puzzle = f.read()

print(f"Part I: {fewest_tokens(puzzle)}")

Part I: 29517


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>29517</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you go to win the first prize, you discover that the claw is nowhere near where you expected it would be. Due to a unit conversion error in your measurements, the position of every prize is actually <code>10000000000000</code> higher on both the <code>X</code> and <code>Y</code> axis!</p>
<p>Add <code>10000000000000</code> to the <code>X</code> and <code>Y</code> position of every prize. After making this change, the example above would now look like this:</p>
<pre><code>Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=10000000008400, Y=10000000005400<br/>
Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=10000000012748, Y=10000000012176<br/>
Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=10000000007870, Y=10000000006450<br/>
Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=10000000018641, Y=10000000010279<br/>
</code></pre>
<p>Now, it is only possible to win a prize on the second and fourth claw machines. Unfortunately, it will take <em>many more than <code>100</code> presses</em> to do so.</p>
<p>Using the corrected prize coordinates, figure out how to win as many prizes as possible. <em>What is the fewest tokens you would have to spend to win all possible prizes?</em></p>
</article>


In [46]:
# @test(tests=tests)
def partII_test(s: str) -> int:
    return fewest_tokens(s, delta=10000000000000, debug=False)

In [47]:
print(f"Part II: {fewest_tokens(puzzle, delta=10000000000000)}")

Part II: 103570327981381


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>902742</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
